In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 37.1 MB/s 
     |████████████████████████████████| 163 kB 16.6 MB/s 


In [ ]:
import re
import json
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import EncoderDecoderModel, AutoTokenizer, pipeline
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split

print("GPU Torch Available = {}".format(torch.cuda.is_available()))
print("Torch Version = {}".format(torch.__version__))

GPU Torch Available = False
Torch Version = 1.12.1+cu113


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading Dataset
file = '/content/drive/MyDrive/Title Generation NLP/Dataset/Dataset_Title_v2.xlsx' 
df = pd.read_excel(file, names = ['Abstract', 'Domain_Labels', 'Title'])
df = df.drop(['Domain_Labels'], axis=1)
df

,Abstract,Title
0,We propose an architecture for VQA which util...,Dual Recurrent Attention Units for Visual Que...
1,Recent approaches based on artificial neural ...,Sequential Short Text Classification with Rec...
2,We introduce the multiresolution recurrent ne...,Multiresolution Recurrent Neural Networks An ...
3,Multi task learning is motivated by the obser...,Learning what to share between loosely relate...
4,We present MILABOT a deep reinforcement learn...,A Deep Reinforcement Learning Chatbot
...,...,...
40995,We study the complexity of learning and appro...,Nearly Tight Bounds on ell Approximation of S...
40996,We consider the problem of multiple users tar...,Concurrent bandits and cognitive radio networks
40997,In this paper we compare and analyze clusteri...,A Comparison of Clustering and Missing Data M...
40998,Cylindrical algebraic decomposition CAD is a ...,Applying machine learning to the problem of c...


In [ ]:
# Use this code to chop off the dataset into 5 or 6 or 7 parts and summarize
parts_df = df[:1000]

In [ ]:
# Summarizer Initialization
summarizer = pipeline(task = "summarization", model = "t5-base", tokenizer= "t5-base", framework = 'pt')

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
# Data Summarization (keep max_length = 100 despite of warnings)
train_abstracts = list(parts_df['Abstract'])
summarized_abstracts = []
counter = 0
for text in train_abstracts:
  print('Summarized {} Abstracts from Data'.format(counter+1))
  summarized_text = summarizer(text, min_length = 20, max_length = 100, return_text = True, return_tensors = False)[0]['summary_text']
  summarized_abstracts.append(summarized_text)
  counter += 1

Summarized 1 Abstracts from Data
Summarized 2 Abstracts from Data
Summarized 3 Abstracts from Data
Summarized 4 Abstracts from Data
Summarized 5 Abstracts from Data
Summarized 6 Abstracts from Data
Summarized 7 Abstracts from Data
Summarized 8 Abstracts from Data
Summarized 9 Abstracts from Data


Your max_length is set to 100, but you input_length is only 88. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


Summarized 10 Abstracts from Data
Summarized 11 Abstracts from Data
Summarized 12 Abstracts from Data
Summarized 13 Abstracts from Data
Summarized 14 Abstracts from Data
Summarized 15 Abstracts from Data
Summarized 16 Abstracts from Data
Summarized 17 Abstracts from Data
Summarized 18 Abstracts from Data
Summarized 19 Abstracts from Data
Summarized 20 Abstracts from Data
Summarized 21 Abstracts from Data
Summarized 22 Abstracts from Data
Summarized 23 Abstracts from Data
Summarized 24 Abstracts from Data
Summarized 25 Abstracts from Data
Summarized 26 Abstracts from Data
Summarized 27 Abstracts from Data
Summarized 28 Abstracts from Data
Summarized 29 Abstracts from Data
Summarized 30 Abstracts from Data
Summarized 31 Abstracts from Data
Summarized 32 Abstracts from Data
Summarized 33 Abstracts from Data
Summarized 34 Abstracts from Data
Summarized 35 Abstracts from Data


Your max_length is set to 100, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


Summarized 36 Abstracts from Data
Summarized 37 Abstracts from Data
Summarized 38 Abstracts from Data
Summarized 39 Abstracts from Data
Summarized 40 Abstracts from Data
Summarized 41 Abstracts from Data
Summarized 42 Abstracts from Data
Summarized 43 Abstracts from Data
Summarized 44 Abstracts from Data
Summarized 45 Abstracts from Data
Summarized 46 Abstracts from Data
Summarized 47 Abstracts from Data
Summarized 48 Abstracts from Data
Summarized 49 Abstracts from Data
Summarized 50 Abstracts from Data
Summarized 51 Abstracts from Data
Summarized 52 Abstracts from Data
Summarized 53 Abstracts from Data
Summarized 54 Abstracts from Data
Summarized 55 Abstracts from Data
Summarized 56 Abstracts from Data
Summarized 57 Abstracts from Data
Summarized 58 Abstracts from Data
Summarized 59 Abstracts from Data
Summarized 60 Abstracts from Data
Summarized 61 Abstracts from Data


...

Summarized 940 Abstracts from Data
Summarized 941 Abstracts from Data
Summarized 942 Abstracts from Data
Summarized 943 Abstracts from Data
Summarized 944 Abstracts from Data
Summarized 945 Abstracts from Data
Summarized 946 Abstracts from Data
Summarized 947 Abstracts from Data
Summarized 948 Abstracts from Data
Summarized 949 Abstracts from Data
Summarized 950 Abstracts from Data
Summarized 951 Abstracts from Data
Summarized 952 Abstracts from Data
Summarized 953 Abstracts from Data
Summarized 954 Abstracts from Data
Summarized 955 Abstracts from Data
Summarized 956 Abstracts from Data
Summarized 957 Abstracts from Data
Summarized 958 Abstracts from Data
Summarized 959 Abstracts from Data
Summarized 960 Abstracts from Data
Summarized 961 Abstracts from Data
Summarized 962 Abstracts from Data
Summarized 963 Abstracts from Data
Summarized 964 Abstracts from Data
Summarized 965 Abstracts from Data
Summarized 966 Abstracts from Data
Summarized 967 Abstracts from Data
Summarized 968 Abstr

Your max_length is set to 100, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


Summarized 992 Abstracts from Data
Summarized 993 Abstracts from Data


Your max_length is set to 100, but you input_length is only 89. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


Summarized 994 Abstracts from Data
Summarized 995 Abstracts from Data
Summarized 996 Abstracts from Data
Summarized 997 Abstracts from Data
Summarized 998 Abstracts from Data
Summarized 999 Abstracts from Data
Summarized 1000 Abstracts from Data


In [ ]:
# Data Preparation into Pandas Dataframe for Model Input
def get_data(dataframe,sum_abs):
  title = list(dataframe['Title'])
  
  raw_data_train = {'Abstract': sum_abs, 'Title': title}
  df = pd.DataFrame(raw_data_train, columns = ['Abstract','Title'])
  return df

data = get_data(parts_df, summarized_abstracts)
data

,Abstract,Title
0,architecture utilizes recurrent layers to gene...,Dual Recurrent Attention Units for Visual Que...
1,many short texts occur in sequences example se...,Sequential Short Text Classification with Rec...
2,the proposed model extends the sequence to seq...,Multiresolution Recurrent Neural Networks An ...
3,humans do not consciously decide to transfer k...,Learning what to share between loosely relate...
4,MILABOT is a deep reinforcement learning chatb...,A Deep Reinforcement Learning Chatbot
...,...,...
995,we use tiled CNNs on standard datasets to lear...,Imaging Time Series to Improve Classification...
996,Blocks is based on Theano a linear algebra com...,Blocks and Fuel Frameworks for deep learning
997,this paper proposes a set of new error criteri...,Adaptive Normalized Risk Averting Training Fo...
998,restricted Boltzmann machines have been shown ...,Training Restricted Boltzmann Machines via th...


In [ ]:
data.to_excel("Summarized_Dataset.xlsx")

In [ ]:
from google.colab import files
def Dataset_Saver(df,filename):
  df.to_excel(filename,index=False) 
  files.download(filename)


In [ ]:
Dataset_Saver(data,"Summarized_Dataset.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>